In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

df = pd.read_csv("european_capitals_history_clean.csv")

n_input = 90  # dni wejściowe
n_output = 7  # dni do predykcji
weather_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres']  # cechy wejściowe

X_all, y_all, cities_all = [], [], []

for city, group in df.groupby('city'):
    group = group.sort_values('date')
    data_X = group[weather_cols].values  # wszystkie cechy jako wejście
    data_y = group[['tavg']].values  # tylko tavg jako wyjście
    for i in range(len(group) - n_input - n_output + 1):
        X_all.append(data_X[i:i + n_input])
        y_all.append(data_y[i + n_input:i + n_input + n_output])
        cities_all.append(city)

X_all = np.array(X_all)  # shape: (samples, n_input, features)
y_all = np.array(y_all)  # shape: (samples, n_output, 1)
cities_all = np.array(cities_all)

print(f"X shape: {X_all.shape}, y shape: {y_all.shape}, liczba próbek: {len(X_all)}")

n_features = X_all.shape[2]

# wejście
X_reshaped = X_all.reshape(-1, n_features)
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X_reshaped).reshape(X_all.shape)

# wyjście
y_reshaped = y_all.reshape(-1, 1)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y_reshaped).reshape(y_all.shape)

model = Sequential()
model.add(LSTM(128, activation='tanh', input_shape=(n_input, n_features)))
model.add(Dropout(0.2))
model.add(Dense(n_output))  # 7 dni wyjścia
model.add(Reshape((n_output, 1)))  # shape (7,1)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

checkpoint = ModelCheckpoint("alstm_weather_best_all_features.h5", monitor='val_loss', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

history = model.fit(
    X_scaled, y_scaled,
    epochs=20,
    batch_size=64,
    validation_split=0.1,
    callbacks=[checkpoint, early_stop]
)

model.save("alstm_weather_final_all_features.h5")
print("✅ Model zapisany jako alstm_weather_final_all_features.h5")

city_name = "Rzym"
city_data = df[df['city'] == city_name].sort_values('date')

# ostatnie 90 dni jako wejście
X_input = city_data[weather_cols].values[-n_input:]
X_input_scaled = scaler_X.transform(X_input).reshape(1, n_input, n_features)

# predykcja
y_pred = model.predict(X_input_scaled)  # shape (1,7,1)
y_pred_actual = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()

# generowanie dat
future_dates = pd.date_range(
    start=pd.to_datetime(city_data['date'].iloc[-1]) + pd.Timedelta(days=1),
    periods=n_output,
    freq='D'
)

# DataFrame z prognozą
df_forecast = pd.DataFrame({
    'date': future_dates,
    'tavg_pred': y_pred_actual,
    'city': city_name
})

print(f"📅 Prognoza 7-dniowa średniej temperatury dla miasta: {city_name}")
print(df_forecast)


X shape: (217406, 90, 6), y shape: (217406, 7, 1), liczba próbek: 217406


C:\Users\gduls\PycharmProjects\pythonProject27\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 128)            │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           903 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 7, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,023 (273.53 KB)

...

3058/3058 ━━━━━━━━━━━━━━━━━━━━ 127s 41ms/step - loss: 0.0023 - mae: 0.0368 - val_loss: 0.0026 - val_mae: 0.0388
Epoch 16/20
3058/3058 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0023 - mae: 0.0368
Epoch 16: val_loss improved from 0.00257 to 0.00253, saving model to alstm_weather_best_all_features.h5


3058/3058 ━━━━━━━━━━━━━━━━━━━━ 126s 41ms/step - loss: 0.0023 - mae: 0.0368 - val_loss: 0.0025 - val_mae: 0.0385
Epoch 17/20
3058/3058 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0023 - mae: 0.0367
Epoch 17: val_loss improved from 0.00253 to 0.00252, saving model to alstm_weather_best_all_features.h5


3058/3058 ━━━━━━━━━━━━━━━━━━━━ 125s 41ms/step - loss: 0.0023 - mae: 0.0367 - val_loss: 0.0025 - val_mae: 0.0385
Epoch 18/20
3057/3058 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0023 - mae: 0.0367
Epoch 18: val_loss did not improve from 0.00252
3058/3058 ━━━━━━━━━━━━━━━━━━━━ 125s 41ms/step - loss: 0.0023 - mae: 0.0366 - val_loss: 0.0025 - val_mae: 0.0385
Epoch 19/20
3058/3058 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0023 - mae: 0.0366
Epoch 19: val_loss did not improve from 0.00252
3058/3058 ━━━━━━━━━━━━━━━━━━━━ 126s 41ms/step - loss: 0.0023 - mae: 0.0366 - val_loss: 0.0025 - val_mae: 0.0387
Epoch 20/20
3057/3058 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0023 - mae: 0.0365
Epoch 20: val_loss improved from 0.00252 to 0.00248, saving model to alstm_weather_best_all_features.h5


3058/3058 ━━━━━━━━━━━━━━━━━━━━ 126s 41ms/step - loss: 0.0023 - mae: 0.0365 - val_loss: 0.0025 - val_mae: 0.0382
Restoring model weights from the end of the best epoch: 20.


✅ Model zapisany jako alstm_weather_final_all_features.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
📅 Prognoza 7-dniowa średniej temperatury dla miasta: Rzym
        date  tavg_pred  city
0 2025-11-06   3.871620  Rzym
1 2025-11-07   4.009537  Rzym
2 2025-11-08   3.869038  Rzym
3 2025-11-09   3.718686  Rzym
4 2025-11-10   3.562924  Rzym
5 2025-11-11   3.348845  Rzym
6 2025-11-12   3.161098  Rzym


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
📅 Prognoza 7-dniowa średniej temperatury dla miasta: Rzym
        date  tavg_pred  city
0 2025-11-06  12.482720  Rzym
1 2025-11-07  13.431561  Rzym
2 2025-11-08  14.161098  Rzym
3 2025-11-09  14.554823  Rzym
4 2025-11-10  14.798719  Rzym
5 2025-11-11  14.867167  Rzym
6 2025-11-12  15.173553  Rzym


In [ ]:
ggg